In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv('../input/train.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
# посмотрим, где есть пропущенные значения
data.isnull().sum()

Некоторые значения о возрасте отсутствуют, надо их как-то заполнить, но заполнять нулями не хочется. Давайте заполним просто средним значением возраста (что, я думаю, тоже не совсем верно)

Некоторые значения Cabin и Embarked тоже не заполнены.

In [ ]:
# посомтрим, сколько людей выжило
sns.countplot('Survived',data=data)
plt.show()

In [ ]:
# Теперь посмотрим, сколько выжило женщин/мужчин
data.groupby(['Sex','Survived'])['Survived'].count()

In [ ]:
# Далее надо все-таки выяснить, можно ли за деньги купить себе жизнь
# Построим график того, сколько людей из какого класса выжило
sns.countplot('Pclass',hue='Survived',data=data)
plt.show()

In [ ]:
# Исходя из верхнего графика видно, что выживших пассажиров первого класса больше, нежели погибших.
# Давайте посмотрим, сколько пассажиров какого класса было и найдем процент выживших среди каждого класса
sns.countplot('Pclass',data=data)
plt.show()

In [ ]:
# Посмотрим на процентное соотношение выживших и умерших по каждому классу
data.groupby(['Pclass','Survived'])['Survived'].count()/data.groupby(['Pclass']).count()['Survived']

Посмотрев на таблицу выше, приходим к печальному (или нет, для кого как) выводу: за деньги все же можно купить себе жизнь. Значит, все-таки пассажиры первого класса были "важнее", чем третьего:(
Цена на билет значения не имеет, но имеет значение класс

In [ ]:
# Теперь давайте посмотрим, сколько людей в каком порту село и сколько из них выжило
sns.countplot('Embarked',hue='Survived',data=data)
plt.show()

Отсюда видно, что больше всех людей село в порте S и выжило оттуда же, но видно забавную закономерность: количество выживших, севших в порте C больше(в процентном соотношении), чем умерших:

In [ ]:
data.groupby(['Embarked','Survived'])['Survived'].count()/data.groupby(['Embarked']).count()['Survived']

Кажется, где-то уже было такое
Давайте посмотрим, связана ли такая высокая выживаемость с полом или классом:

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,5))
sns.countplot('Embarked',hue='Sex',data=data,ax=ax[0])
ax[0].set_title(u'Распредление людей по полу')
sns.countplot('Embarked',hue='Pclass',data=data,ax=ax[1])
ax[1].set_title(u'Распределение людей по классу')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

In [ ]:
data.groupby(['Embarked','Survived', 'Pclass'])['Survived'].count()

In [ ]:
data.Age.fillna(data.Age.mean(), inplace=True)
data.Age.isnull().any()

In [ ]:
data.Embarked.fillna('S', inplace=True)
data.Embarked.isnull().any()

Теперь надо бы дропнуть категориальные и ненужные признаки, руководствуясь тем, что, в принципе перед лицом катастрофы плевать, за сколкьо ты купил билет, в какой каюте должен был находиться (ключевое "должен", потому что, вообще говоря, признак очень неустойчивый), id, то, в каком порту сел на корабль, наверное, никак тоже не влияет на выживаемость

In [ ]:
data.drop(['Name','Ticket','Fare','Cabin','PassengerId'],axis=1,inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)

In [ ]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)

Давайте теперь что-нибудь предскажем, с помощью решающего дерева и KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 

In [ ]:
train,test=train_test_split(data,test_size=0.3,random_state=42,stratify=data['Survived'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['Survived']

In [ ]:
dec_tree=DecisionTreeClassifier()
dec_tree.fit(train_X,train_Y)
DT_prediction=dec_tree.predict(test_X)
print(metrics.accuracy_score(DT_prediction,test_Y))

In [ ]:
KNN=KNeighborsClassifier(n_neighbors=7) 
KNN.fit(train_X,np.ravel(train_Y))
KNN_prediction=KNN.predict(test_X)
test_Y = np.ravel(test_Y)
print(metrics.accuracy_score(KNN_prediction,test_Y))